# Submitting calculations
#### Create a structure, kpoints, and input parameters and submit a Quantum ESPRESSO (PW) calculation
Time: 3 mins

In [ ]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm.utils import load_node
from aiida.work.run import run
from aiida.orm.code import Code
from aiida.orm.data.base import Bool, Str
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.structure import StructureData
from ase.lattice.spacegroup import crystal

import nglview
#import warnings
#warnings.filterwarnings('ignore')

#### Create a diamond cubic crystal structure

In [ ]:
# Define a structure, employing the Atomic Structure Environment library
alat = 3.568
ase_definition = crystal('C', [(0,0,0)], spacegroup=227, cellpar=[alat, alat, alat, 90, 90, 90], primitive_cell=True)*2
structure = StructureData(ase=ase_definition)
structure.store()

#### Show the already stored structure

In [ ]:
view = nglview.show_ase(structure.get_ase())
view

#### Create the _k_-points mesh

In [ ]:
# Define the k-points mesh
kpoints = KpointsData()
kpoints_mesh = [1, 1, 1]
kpoints.set_kpoints_mesh(kpoints_mesh)
kpoints.store()

#### List existing pseudopotential families

In [ ]:
!verdi data upf listfamilies

#### Define the calculation input parameters

In [ ]:
# Define the pseudo potential family and input parameters for pw.x
pseudo_family = 'SSSP'
parameters = {
    'CONTROL': {
        "calculation": "scf",
        'tstress': True,
    },
    'SYSTEM': {
        'ecutwfc': 40.,
        'ecutrho': 320.,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

#### See the available codes and select one of them

In [ ]:
!verdi code list

#### Get the Code instance from the name

In [ ]:
codename = 'pw-5.1@localhost'
code = Code.get_from_string(codename)

#### Create a new calculation from the code and set the input parameters

In [ ]:
calc = code.new_calc()
calc.use_structure(structure)
calc.use_pseudos_from_family(pseudo_family)
calc.use_parameters(ParameterData(dict=parameters))
calc.use_kpoints(kpoints)
calc.set_resources({"num_machines": 1})
calc.set_max_wallclock_seconds(30*60)

#### Store the calculation and submit it to the daemon

In [ ]:
calc.store_all()
calc.submit()
print "Submitted a {} with pk<{}>".format(calc.__class__.__name__, calc.pk)

#### Check the status of the calculation

In [ ]:
!verdi calculation list

#### Monitor its status until completion

In [ ]:
import time
from IPython.display import clear_output

def check_calculation_status(calc):
    while not calc.has_finished():
        clear_output()
        !verdi calculation list
        time.sleep(2)

    clear_output()
    !verdi calculation list $calc.pk

In [ ]:
check_calculation_status(calc)

#### Check that the calculation finished successfully

In [ ]:
calc.has_finished_ok()

#### We can easily show all the attached outputs

In [ ]:
calc.get_outputs_dict()

#### There are convenient methods to directly access the results

In [ ]:
print "Total energy = {} {}".format(calc.res.energy, calc.res.energy_units)